# QAOA 
Reference: https://pennylane.ai/qml/demos/tutorial_qaoa_maxcut

In [1]:
import pennylane as qml
from pennylane import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import time
import pandas as pd
import seaborn as sns
import os
from datetime import datetime
from pennylane import qaoa

## Graph generating function

In [2]:
def generate_random_graph(n_nodes=8, p=0.5, seed=None):
    G = nx.erdos_renyi_graph(n_nodes, p, seed=seed)
    for (u, v) in G.edges():
        G.edges[u, v]['weight'] = int(np.random.randint(1, 10))   
    G = nx.convert_node_labels_to_integers(G)
    return G


## Algorithm function

In [3]:
def maxcut_qaoa(G, p_layers=1, optimizer_steps=50, seed=None):
    np.random.seed(seed)
    n = G.number_of_nodes()
    edges = list(G.edges())
    dev = qml.device("default.qubit", wires=n, shots=100)

    # Build cost Hamiltonian using PennyLane's built-in function
    cost_h, _ = qaoa.maxcut(G)

    def qaoa_layer(gamma, beta):
        # Cost layer
        for (i, j) in edges:
            qml.CNOT(wires=[i, j])
            qml.RZ(-gamma, wires=j)
            qml.CNOT(wires=[i, j])
        # Mixer layer
        for i in range(n):
            qml.RX(2 * beta, wires=i)

    @qml.qnode(dev)
    def circuit(params):
        # Initial state
        for i in range(n):
            qml.Hadamard(wires=i)
        # QAOA layers
        for l in range(p_layers):
            qaoa_layer(params[l, 0], params[l, 1])
        return qml.expval(cost_h)

    @qml.qnode(dev)
    def sample_circuit(params):
        for i in range(n):
            qml.Hadamard(wires=i)
        for l in range(p_layers):
            qaoa_layer(params[l, 0], params[l, 1])
        return [qml.sample(qml.PauliZ(i)) for i in range(n)]

    # Initial parameters
    params = 0.01 * np.random.randn(p_layers, 2)
    params = qml.numpy.array(params, requires_grad=True)
    opt = qml.GradientDescentOptimizer(stepsize=0.1)

    for step in range(optimizer_steps):
        params = opt.step(lambda v: -circuit(v), params)

    # After optimization
    cut_value = circuit(params)
    # Sample bitstrings
    samples = np.array(sample_circuit(params))
    # Convert {-1,1} to {0,1}
    bitstrings = ((1 - samples.T) // 2).astype(int)
    # Evaluate cut for each sample
    def cut_from_bitstring(bitstring):
        return sum(G.edges[i, j]['weight'] for i, j in edges if bitstring[i] != bitstring[j])
    cut_values = [cut_from_bitstring(bs) for bs in bitstrings]
    max_idx = np.argmax(cut_values)
    best_bitstring = bitstrings[max_idx]
    best_cut_value = cut_values[max_idx]
    
    return best_bitstring, best_cut_value

## Experiment function

In [4]:
def run_qaoa_experiment_time_budget(sizes, edge_prob=0.5, time_per_size=420, p_layers=1, optimizer_steps=50):
    all_cut_values = []

    for n_nodes in sizes:
        cut_values = []
        start_time = time.time()
        g_idx = 0

        while time.time() - start_time < time_per_size:
            G = generate_random_graph(n_nodes=n_nodes, p=edge_prob, seed=g_idx)
            _, value = maxcut_qaoa(G, p_layers=p_layers, optimizer_steps=optimizer_steps, seed=g_idx)
            cut_values.append(value)
            g_idx += 1
        all_cut_values.append(cut_values)
        print(f"Size: {n_nodes} | Graphs: {g_idx} | Mean Cut: {np.mean(cut_values):.2f} | Std: {np.std(cut_values):.2f}")
    
    return all_cut_values

## Experiment and results save

In [5]:
sizes = [4, 6, 8, 10]
edge_prob = 0.5
time_per_size = 900
p_layers = 1
optimizer_steps = 50

cut_values = run_qaoa_experiment_time_budget(sizes, edge_prob, time_per_size, p_layers, optimizer_steps)

df = pd.DataFrame({
    'Graph Size': np.repeat(sizes, [len(cut) for cut in cut_values]),
    'Cut Value': np.concatenate(cut_values)
})

# Save csv
os.makedirs('data/qaoa-data', exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
csv_path = f'data/qaoa-data/qaoa_{timestamp}.csv'
df.to_csv(csv_path, index=False)
print(f"Results saved to {csv_path}")

# Save plot
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
plot_path = f'plots/qaoa-plots/qaoa_violin_{timestamp}.png'

plt.figure(figsize=(12, 6))
sns.violinplot(x='Graph Size', y='Cut Value', data=df, inner='quartile')
plt.title('QAOA Max-Cut Values by Graph Size')
plt.xlabel('Graph Size')
plt.ylabel('Cut Value')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(plot_path)
print(f"Plot saved to {plot_path}")
plt.show()

TypeError: Can't differentiate w.r.t. type <class 'numpy.int64'>